In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install pydot --quiet
!pip install tensorflow==2.15.0 --quiet
!pip install transformers --quiet
!pip install sentencepiece --quiet
!pip install datasets --quiet
!pip install accelerate --quiet
!pip install transformers[torch] --quiet
!pip install evaluate --quiet
!pip install rouge_score --quiet
!pip install jiwer --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
import transformers
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import pandas as pd
import accelerate
import torch
from torch.utils.data import Dataset
from evaluate import load
pd.set_option('display.max_colwidth', 0)

In [ ]:
# ====== ENVIRONMENT ======
DEV = False
EXPLORE = False
RANDOM_SEED = 42

# ====== DATA =======
INPUT_MAX_LENGTH = 1024
LABEL_MAX_LENGTH = 128

# ====== MODEL ======
CHECKPOINT = "facebook/bart-large"
PATH = '/content/drive/MyDrive/colab-notebooks/W266/'
PEFT = False

# ====== OPTIIZER =======
OPTIMIZER = "adamw_bnb_8bit"
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01

# ====== TRAINING ======
EVAL_STRATEGY = "steps"
BATCH_SIZE = 6
EPOCHS = 1
SAVE_STEPS = 100 if DEV else 1000
EVAL_STEPS = 100 if DEV else 250
LOGGING_STEPS = 100 if DEV else 500
METRIC_WER = load("wer")
METRIC_ROUGE = load("rouge")

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/W266/huggingface_credentials.txt", "r") as f:
    HF_TOKEN = f.read()


In [ ]:
df = load_dataset("jordanfan/processed_us_congress_117_bills")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11277 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/3388 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/377 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
def model_init():
  seed = 42
  torch.manual_seed(seed)
  return AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)

#model = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def preprocess_data_baseline(df):
  input = [doc for doc in df["cur_text"]]
  #tokenize full text
  model_inputs = tokenizer(input, max_length=INPUT_MAX_LENGTH, return_tensors = "pt", padding=True, truncation=True)
  #tokenize the summaries
  summary_input = [doc for doc in df["cleaned_summary"]]
  targets = tokenizer(text_target = summary_input, max_length=LABEL_MAX_LENGTH, padding= True, truncation=True)
  #set labels
  model_inputs['labels'] = targets['input_ids']
  return model_inputs


In [ ]:
tokenized_data_baseline = df.map(preprocess_data_baseline, batched = True)

Map:   0%|          | 0/11277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3388 [00:00<?, ? examples/s]

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=CHECKPOINT)

In [ ]:
def compute_metrics(pred):

    labels_ids = pred.label_ids
    pred_ids = pred.predictions[0]

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    result_wer = METRIC_WER.compute(predictions=pred_str, references=label_str)
    result_rouge = METRIC_ROUGE.compute(predictions=pred_str, references=label_str, use_stemmer=True, use_aggregator=True)
    result_rouge = {key: value for key, value in result_rouge.items()}

    results = {key: value for key, value in result_rouge.items()}
    results["WER"] = result_wer
    return {k: round(v, 4) for k,v in results.items()}

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir = f"jordanfan/bart_baseline_{INPUT_MAX_LENGTH}",
    evaluation_strategy=EVAL_STRATEGY,
    save_strategy=EVAL_STRATEGY,
    save_steps=SAVE_STEPS,
    eval_steps=EVAL_STEPS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
#    eval_accumulation_steps=16,
    load_best_model_at_end=True,
    num_train_epochs=EPOCHS,
    # predict_with_generate=True,
    fp16=True,
    report_to=["tensorboard"],
    push_to_hub=True,
    hub_token=HF_TOKEN
    )

trainer = Seq2SeqTrainer(
    #model = model,
    model_init=model_init,
    args = args,
    train_dataset=tokenized_data_baseline["train"],
    eval_dataset=tokenized_data_baseline["val"],
    tokenizer=tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Wer
250,No log,1.218040,0.660100,0.388200,0.595400,0.595200,0.498100
500,2.109900,1.115236,0.680000,0.413300,0.616500,0.616400,0.480600
750,2.109900,1.076512,0.681700,0.417700,0.618100,0.618000,0.472300
1000,1.185300,1.055206,0.691500,0.428300,0.628400,0.628300,0.466400
1250,1.185300,1.031547,0.695800,0.432700,0.632800,0.632600,0.462200
1500,1.117900,1.022687,0.691200,0.431700,0.629700,0.629600,0.458200
1750,1.117900,1.010009,0.696100,0.436900,0.635200,0.635200,0.456200


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1880, training_loss=1.398263923158037, metrics={'train_runtime': 2813.8523, 'train_samples_per_second': 4.008, 'train_steps_per_second': 0.668, 'total_flos': 2.4438438598017024e+16, 'train_loss': 1.398263923158037, 'epoch': 1.0})

In [ ]:
trainer.create_model_card(
    language='english',
    model_name=f"bart-baseline-{INPUT_MAX_LENGTH}",
    tasks='summarization',
    tags='summarization',
    dataset='jordanfan/processed_us_congress_117_bills',
    dataset_args=f"Max token input: {INPUT_MAX_LENGTH} | {LABEL_MAX_LENGTH}"
)

commit_msg = f"training completed[dev]: {INPUT_MAX_LENGTH} {LABEL_MAX_LENGTH}"
trainer.push_to_hub(commit_message=commit_msg)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


events.out.tfevents.1711086530.9164c72d17ee.1634.0:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jordanfan/bart_baseline_1024/commit/884e867a2c2f18536cd37ea290e71ec0f764b4fd', commit_message='training completed[dev]: 1024 128', commit_description='', oid='884e867a2c2f18536cd37ea290e71ec0f764b4fd', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
predict_results = trainer.predict(tokenized_data_baseline["test"])
trainer.save_metrics(f"test-{INPUT_MAX_LENGTH}", predict_results.metrics)